In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from prophet import Prophet
import matplotlib.pyplot as plt
import sys

In [3]:
print("Python version:", sys.version)
print("XGBoost version:", xgb.__version__)
print("Pandas version:", pd.__version__)

Python version: 3.12.3 (tags/v3.12.3:f6650f9, Apr  9 2024, 14:05:25) [MSC v.1938 64 bit (AMD64)]
XGBoost version: 2.1.2
Pandas version: 2.2.2


In [45]:
# Load and prepare data
file_path = r"C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Code\Aditya\TCS_Sorted_Quarterly_Data.xlsx"
df = pd.read_excel(file_path, sheet_name="Sheet1")
df.head(3)

,Quarterly Results of Tata Consultancy Services(in Rs. Cr.),Net sales/income from operations,Total income from operations,Employees cost,depreciat,Other expenses,"P/l before other inc. , int., excpt. items & tax",Other income,"P/l before int., excpt. items & tax",Interest,...,Tax,P/l after tax from ordinary activities,Net profit/(loss) for the period,Equity share capital,Basic eps,Diluted eps,Basic eps.,Diluted eps.,Total Revenue,Total Expenditure
0,Mar 05 Q4,2147.66,0.0,470.36,42.64,1122.13,0.0,-30.33,0.0,1.59,...,53.91,0.0,426.70,48.01,0.0,0.0,0.0,0.0,2117.33,1690.63
1,Jun 05 Q1,2250.06,0.0,504.28,46.61,1052.55,0.0,26.26,0.0,0.59,...,93.43,0.0,578.86,48.01,0.0,0.0,0.0,0.0,2276.32,1697.46
2,Sep 05 Q2,2504.49,0.0,989.37,56.33,759.09,0.0,11.24,0.0,0.08,...,90.54,0.0,620.32,48.01,0.0,0.0,0.0,0.0,2515.73,1895.41


In [46]:
# Rename columns
df.rename(columns={
    "Quarterly Results of Tata Consultancy Services(in Rs. Cr.)": "ds",
    "Net profit/(loss) for the period": "y",
    "Total Revenue": "revenue",
    "Total Expenditure": "total_expenditure"
}, inplace=True)
df.head(3)

,ds,Net sales/income from operations,Total income from operations,Employees cost,depreciat,Other expenses,"P/l before other inc. , int., excpt. items & tax",Other income,"P/l before int., excpt. items & tax",Interest,...,Tax,P/l after tax from ordinary activities,y,Equity share capital,Basic eps,Diluted eps,Basic eps.,Diluted eps.,revenue,total_expenditure
0,Mar 05 Q4,2147.66,0.0,470.36,42.64,1122.13,0.0,-30.33,0.0,1.59,...,53.91,0.0,426.70,48.01,0.0,0.0,0.0,0.0,2117.33,1690.63
1,Jun 05 Q1,2250.06,0.0,504.28,46.61,1052.55,0.0,26.26,0.0,0.59,...,93.43,0.0,578.86,48.01,0.0,0.0,0.0,0.0,2276.32,1697.46
2,Sep 05 Q2,2504.49,0.0,989.37,56.33,759.09,0.0,11.24,0.0,0.08,...,90.54,0.0,620.32,48.01,0.0,0.0,0.0,0.0,2515.73,1895.41


In [47]:
def parse_quarter(date_str):
    month_map = {"Mar": ("03", "31"), "Jun": ("06", "30"), "Sep": ("09", "30"), "Dec": ("12", "31")}
    parts = date_str.split()
    year = "20" + parts[1] if int(parts[1]) < 25 else "19" + parts[1]
    month, day = month_map[parts[0]]
    return pd.to_datetime(f"{year}-{month}-{day}")

df["ds"] = df["ds"].apply(parse_quarter)
df.head(3)

,ds,Net sales/income from operations,Total income from operations,Employees cost,depreciat,Other expenses,"P/l before other inc. , int., excpt. items & tax",Other income,"P/l before int., excpt. items & tax",Interest,...,Tax,P/l after tax from ordinary activities,y,Equity share capital,Basic eps,Diluted eps,Basic eps.,Diluted eps.,revenue,total_expenditure
0,2005-03-31,2147.66,0.0,470.36,42.64,1122.13,0.0,-30.33,0.0,1.59,...,53.91,0.0,426.70,48.01,0.0,0.0,0.0,0.0,2117.33,1690.63
1,2005-06-30,2250.06,0.0,504.28,46.61,1052.55,0.0,26.26,0.0,0.59,...,93.43,0.0,578.86,48.01,0.0,0.0,0.0,0.0,2276.32,1697.46
2,2005-09-30,2504.49,0.0,989.37,56.33,759.09,0.0,11.24,0.0,0.08,...,90.54,0.0,620.32,48.01,0.0,0.0,0.0,0.0,2515.73,1895.41


In [48]:
# Feature engineering
df["trend"] = np.arange(len(df))
for lag in [1, 2, 3, 4]:
    df[f"lag_{lag}"] = df["y"].shift(lag)
df["rolling_mean"] = df["y"].rolling(window=4, min_periods=1).mean()
df["rolling_std"] = df["y"].rolling(window=4, min_periods=1).std()
df["revenue_expenditure"] = df["revenue"] * df["total_expenditure"]
df.to_excel(r"C:\Users\sharm\OneDrive\Desktop\Kishan\Data\Hybrid Data\lrmrsre.xlsx",index=False)

In [49]:
# Handle missing values
df = df.ffill().bfill() 
df.to_excel(r"C:\Users\sharm\OneDrive\Desktop\Kishan\Data\Hybrid Data\fbfill_mv.xlsx",index=False)

In [50]:
# Cap outliers
q1, q3 = np.percentile(df["y"], [25, 75])
iqr = q3 - q1
lower_bound, upper_bound = q1 - 1.5 * iqr, q3 + 1.5 * iqr
print(f"Outlier bounds: ({lower_bound:.2f}, {upper_bound:.2f})")
outliers = df["y"].apply(lambda x: x < lower_bound or x > upper_bound).sum()
print(f"Found {outliers} outliers in target variable")
df["y"] = df["y"].clip(lower_bound, upper_bound)
df.to_excel(r"C:\Users\sharm\OneDrive\Desktop\Kishan\Data\Hybrid Data\3_q1q3_outliers.xlsx",index=False)

Outlier bounds: (-8051.83, 17488.30)
Found 0 outliers in target variable


In [51]:
# Train-test split
train_size = int(0.8 * len(df))
train_df = df.iloc[:train_size].copy()
test_df = df.iloc[train_size:].copy()
dates_train, dates_test = train_df["ds"], test_df["ds"]

In [52]:
# Prophet model
print("Training models...")
prophet_df = train_df[["ds", "y"]].copy()
prophet_df["revenue"] = train_df["revenue"]
prophet_df["total_expenditure"] = train_df["total_expenditure"]
prophet_df.to_excel(r"C:\Users\sharm\OneDrive\Desktop\Kishan\Data\Hybrid Data\4_Prophet_df.xlsx",index=False)

Training models...


In [53]:
prophet_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False,
    seasonality_mode="additive"
)

In [54]:
prophet_model.add_regressor("revenue")
prophet_model.add_regressor("total_expenditure")
prophet_model.fit(prophet_df)

11:21:17 - cmdstanpy - INFO - Chain [1] start processing
11:21:17 - cmdstanpy - INFO - Chain [1] done processing


In [55]:
# Prophet predictions
prophet_train_pred = prophet_model.predict(prophet_df)
prophet_train_pred.to_excel(r"C:\Users\sharm\OneDrive\Desktop\Kishan\Data\Hybrid Data\5_Prophet_train_pred.xlsx",index=False)
train_df["prophet_pred"] = prophet_train_pred["yhat"].values

In [56]:
prophet_test_df = test_df[["ds", "revenue", "total_expenditure"]].copy()
prophet_test_pred = prophet_model.predict(prophet_test_df)
prophet_test_pred.to_excel(r"C:\Users\sharm\OneDrive\Desktop\Kishan\Data\Hybrid Data\6_Prophet_test_pred.xlsx",index=False)
test_df["prophet_pred"] = prophet_test_pred["yhat"].values

In [44]:
# Compute residuals
train_df["residual"] = train_df["y"] - train_df["prophet_pred"]
test_df["residual"] = test_df["y"] - test_df["prophet_pred"]